In [702]:
import pandas as pd
import datetime
import numpy as np

In [703]:
# importing news article csv files and cleaning them
fb_articles_df = pd.read_csv('fb_articles_20180822_20180829_df.csv', index_col = 0)
fb_articles_df['publishedAt'] = pd.to_datetime(fb_articles_df['publishedAt']) # set dtype to datetime
fb_articles_df['publishedAt'] = fb_articles_df['publishedAt'].dt.round('30min') 
fb_articles_df = fb_articles_df.set_index(['publishedAt'])

# importing tweet csv files and cleaning them
fb_tweets_df = pd.read_csv('tweets_0822_0829_final.csv')
fb_tweets_df['time_stamp'] = pd.to_datetime(fb_tweets_df['time_stamp'])
fb_tweets_df['time_stamp'] = fb_tweets_df['time_stamp'].dt.round('30min')
fb_tweets_df_ = fb_tweets_df.set_index(['time_stamp'])

# reading stock.csv and cleaning the df
running_stocks_thirty = pd.read_csv('stock.csv', index_col = 0)
running_stocks_thirty.index = pd.to_datetime(running_stocks_thirty.index)

# updating stock data
running_stocks_thirty = running_stocks_thirty[running_stocks_thirty.index.day >= 24].sort_index()
running_stocks_thirty['increase_decrease'] = running_stocks_thirty['4. close'] - running_stocks_thirty['1. open']
running_stocks_thirty['up_down'] = np.where(running_stocks_thirty['increase_decrease']>=0, 1, 0)
running_stocks_thirty['movement(%)'] = np.round(((running_stocks_thirty['4. close'] - running_stocks_thirty['1. open'])/running_stocks_thirty['4. close'])*100, 2)

# reading tech indicators
fb_tech_indic = pd.read_csv('fb_tech_indicators.csv', index_col = 0 )
fb_tech_indic.index = pd.to_datetime(fb_tech_indic.index)
fb_tech_indic_ = fb_tech_indic[fb_tech_indic.index.day >= 24]
fb_tech_indic_df = fb_tech_indic_[fb_tech_indic_.index.day < 30]

In [704]:
# drop all the tweets before the 24th
fb_tweets_df = fb_tweets_df_[fb_tweets_df_.index.day >= 24]
fb_tweets_master_df = fb_tweets_df.drop(['t_id', 'text'], axis = 1)

In [705]:
fb_tweets_df_24 = fb_tweets_master_df[fb_tweets_master_df.index.day ==24]
fb_tweets_df_25 = fb_tweets_master_df[fb_tweets_master_df.index.day ==25]
fb_tweets_df_26 = fb_tweets_master_df[fb_tweets_master_df.index.day ==26]
fb_tweets_df_27 = fb_tweets_master_df[fb_tweets_master_df.index.day ==27]
fb_tweets_df_28 = fb_tweets_master_df[fb_tweets_master_df.index.day ==28]
fb_tweets_df_29 = fb_tweets_master_df[fb_tweets_master_df.index.day ==29]

In [706]:
# seperate during/after market hrs
def during_after(df):
    df_during = df.between_time('9:30', "15:30")
    df_after = df.between_time('15:30', "9:30")
    return df_during, df_after

In [707]:
open_hours_tweets_24, after_hour_tweets_24 = during_after(fb_tweets_df_24)
open_hours_tweets_25, after_hour_tweets_25 = during_after(fb_tweets_df_25)
open_hours_tweets_26, after_hour_tweets_26 = during_after(fb_tweets_df_26)
open_hours_tweets_27, after_hour_tweets_27 = during_after(fb_tweets_df_27)
open_hours_tweets_28, after_hour_tweets_28 = during_after(fb_tweets_df_28)
open_hours_tweets_29, after_hour_tweets_29 = during_after(fb_tweets_df_29)

In [708]:
# aggregate the scores 
def aggregate(df):
    sent_avg = df.groupby(df.index).agg({'vader_sentiment': 'mean'}).rename(columns = {'vader_sentiment': 'sentiment_avg_30'}).shift()
    sent_count = df.groupby(df.index).agg({'sentiment': 'value_counts'}).unstack().rename(columns = {'sentiment': 'sentiment_30'}).shift()
    df_ = pd.concat([sent_avg, sent_count], axis = 1)
    df_ini = df_.fillna(df_.mean())
    df_hour_i = df_ini.shift().rolling(window=2, center = True, min_periods=2).mean().rename(columns = {'sentiment_avg_30': 'sentiment_avg_60'})
    df_hour = df_hour_i.fillna(df_hour_i.mean())
    df_two_hours_i = df_ini.shift(2).rolling(window=4, center = True, min_periods=4).mean().rename(columns = {'sentiment_avg_30': 'sentiment_avg_120'})
    df_two_hours = df_two_hours_i.fillna(df_two_hours_i.mean())
    df_all = pd.concat([df_ini, df_hour, df_two_hours], axis = 1)
    return df_all

In [709]:
open_hours_tweets_24 = aggregate(open_hours_tweets_24)
open_hours_tweets_25 = aggregate(open_hours_tweets_25)
open_hours_tweets_26 = aggregate(open_hours_tweets_26)
open_hours_tweets_27 = aggregate(open_hours_tweets_27)
open_hours_tweets_28 = aggregate(open_hours_tweets_28)
open_hours_tweets_29 = aggregate(open_hours_tweets_29)

In [710]:
# seperate night-before and morning-day-of market hrs
def night_morning(df):
    df_night = df.between_time('15:30', "0:00")
    df_morning = df.between_time('0:00', "9:30")
    df_night_avg = df_night.groupby(df_night.index.day).mean()
    df_night_ct = df_night.groupby(df_night.index.day).agg({'sentiment': 'value_counts'}).unstack()
    night_df = pd.concat([df_night_avg, df_night_ct], axis = 1)
    night_df.columns = ['vader_sentiment_night', 'sent_neg, night_before', 'sent_neu, night_before', 'sent_pos, night_before']
    df_morning_avg = df_morning.groupby(df_morning.index.day).mean()
    df_morning_ct = df_morning.groupby(df_morning.index.day).agg({'sentiment': 'value_counts'}).unstack()
    morning_df = pd.concat([df_morning_avg, df_morning_ct], axis = 1)
    morning_df.columns = ['vader_sentiment_morning', 'sent_neg, morning', 'sent_neu, morning', 'sent_pos, morning']
    pd.to_datetime(morning_df.index)
    pd.to_datetime(night_df.index)
    return night_df, morning_df

In [711]:
fb_tweets_df_23 = fb_tweets_df_[fb_tweets_df_.index.day == 23]
fb_tweets_23_df = fb_tweets_df_23.drop(['t_id', 'text'], axis = 1)
fb_tweets_23_night = fb_tweets_23_df.between_time('15:30', "9:30")
after_hour_tweets_24_night, after_hour_tweets_23_morning = night_morning(fb_tweets_23_night)

In [714]:
after_hour_tweets_25_night, after_hour_tweets_24_morning = night_morning(after_hour_tweets_24)
after_hour_tweets_26_night, after_hour_tweets_25_morning = night_morning(after_hour_tweets_25)
after_hour_tweets_27_night, after_hour_tweets_26_morning = night_morning(after_hour_tweets_26)
after_hour_tweets_28_night, after_hour_tweets_27_morning = night_morning(after_hour_tweets_27)
after_hour_tweets_29_night, after_hour_tweets_28_morning = night_morning(after_hour_tweets_28)
after_hour_tweets_30_night, after_hour_tweets_29_morning = night_morning(after_hour_tweets_29)

In [715]:
# update the index of the night df to the next day
after_hour_tweets_24_night.index = [24]
after_hour_tweets_25_night.index = [25]
after_hour_tweets_26_night.index = [26]
after_hour_tweets_27_night.index = [27]
after_hour_tweets_28_night.index = [28]
after_hour_tweets_29_night.index = [29]

In [716]:
night_morning_24 = pd.concat([after_hour_tweets_24_night, after_hour_tweets_24_morning], axis = 1)
night_morning_25 = pd.concat([after_hour_tweets_25_night, after_hour_tweets_25_morning], axis = 1)
night_morning_26 = pd.concat([after_hour_tweets_26_night, after_hour_tweets_26_morning], axis = 1)
night_morning_27 = pd.concat([after_hour_tweets_27_night, after_hour_tweets_27_morning], axis = 1)
night_morning_28 = pd.concat([after_hour_tweets_28_night, after_hour_tweets_28_morning], axis = 1)
night_morning_29 = pd.concat([after_hour_tweets_29_night, after_hour_tweets_29_morning], axis = 1)

In [719]:
list_of_columns = list(night_morning_24.columns)
def concat_main_night_morning(open_hour_df, night_morning_df):
    for i in range(0,len(list_of_columns)):
        open_hour_df[list_of_columns[i]] = np.full((len(open_hour_df),1), night_morning_df[list_of_columns[i]].values[0])

In [720]:
concat_main_night_morning(open_hours_tweets_24, night_morning_24)
concat_main_night_morning(open_hours_tweets_25, night_morning_25)
concat_main_night_morning(open_hours_tweets_26, night_morning_26)
concat_main_night_morning(open_hours_tweets_27, night_morning_27)
concat_main_night_morning(open_hours_tweets_28, night_morning_28)
concat_main_night_morning(open_hours_tweets_29, night_morning_29)

In [721]:
fb_tweets_master = pd.concat([open_hours_tweets_24, open_hours_tweets_25, open_hours_tweets_26, open_hours_tweets_27, open_hours_tweets_28, open_hours_tweets_29])
fb_tweets_master.columns = ['sentiment_avg_30','coun_neg, 30', 'coun_neu, 30','coun_pos, 30', 'sentiment_avg_60', 'coun_neg, 60', 'coun_neu, 60','coun_pos, 60','sentiment_avg_120', 'coun_neg, 120', 'coun_neu, 120','coun_pos, 120', 'sentiment_night_before_avg', 'count_neg, night_before', 'count_neu, night_before', 'count_pos, night_before','sentiment_morning_avg', 'count_neg, morning', 'count_neu, morning', 'count_pos, morning']
# take out weekend
fb_tweets_master = fb_tweets_master[(fb_tweets_master.index.day != 25) & (fb_tweets_master.index.day != 26)]

In [776]:
# put stock and tweet data together
fb_tweets_stocks = pd.concat([running_stocks_thirty, fb_tech_indic_df, fb_tweets_master, fb_articles_df], axis = 1)

In [777]:
fb_tweets_stocks.to_csv('fb_tweets_stock.csv')

In [767]:
fb_articles_df = pd.read_csv('fb_article_avgs.csv', index_col = 0)
fb_articles_df.index = pd.to_datetime(fb_articles_df.index)

In [768]:
fb_articles_df_ = fb_articles_df[fb_articles_df.index.day >= 24]
fb_articles_df = fb_articles_df_[fb_articles_df_.index.day < 30]

In [772]:
fb_articles_df = fb_articles_df.between_time('9:30', "15:30").sort_index()

In [775]:
fb_articles_df

,day,day compound avg,day positive avg,day negative avg,day neutral avg,two day neutral avg,two day positive avg,two day negative avg,two day compound avg,two day nbc-news compound avg,...,one day reuters pos avg,one day wired pos avg,one day techcrunch pos avg,one day nbc-news neu avg,one day business-insider neu avg,one day the-new-york-times neu avg,one day abc-news neu avg,one day reuters neu avg,one day wired neu avg,one day techcrunch neu avg
2018-08-24 09:30:00,2018-08-24,-0.982500,0.087000,0.096000,0.817000,0.854688,0.077563,0.067812,0.166441,-0.288667,...,0.065133,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.879650,0.854750,0.862225
2018-08-24 10:00:00,2018-08-24,-0.982500,0.087000,0.096000,0.817000,0.854688,0.077563,0.067812,0.166441,-0.288667,...,0.065133,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.879650,0.854750,0.862225
2018-08-24 10:30:00,2018-08-24,-0.982500,0.087000,0.096000,0.817000,0.854688,0.077563,0.067812,0.166441,-0.288667,...,0.065133,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.879650,0.854750,0.862225
2018-08-24 11:00:00,2018-08-24,-0.982500,0.087000,0.096000,0.817000,0.854688,0.077563,0.067812,0.166441,-0.288667,...,0.065133,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.879650,0.854750,0.862225
2018-08-24 11:30:00,2018-08-24,-0.982500,0.087000,0.096000,0.817000,0.854688,0.077563,0.067812,0.166441,-0.288667,...,0.065133,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.879650,0.854750,0.862225
2018-08-24 12:00:00,2018-08-24,-0.972500,0.061500,0.094000,0.844000,0.855182,0.076303,0.068545,0.132230,-0.288667,...,0.036000,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.871000,0.854750,0.862225
2018-08-24 12:30:00,2018-08-24,-0.969167,0.053000,0.093333,0.853000,0.855647,0.075118,0.069235,0.100032,-0.288667,...,0.036000,0.078250,0.099175,0.878667,0.871012,0.817000,0.8300,0.871000,0.854750,0.862225
2018-08-24 13:00:00,2018-08-24,-0.972800,0.054000,0.107750,0.837750,0.853829,0.074600,0.071571,0.069069,-0.288667,...,0.036000,0.078250,0.099175,0.878667,0.871012,0.817000,0.7920,0.871000,0.854750,0.862225
2018-08-24 13:30:00,2018-08-24,-0.579240,0.063600,0.096600,0.839400,0.853611,0.075361,0.071028,0.094789,-0.288667,...,0.036000,0.078250,0.099175,0.878667,0.871012,0.831500,0.7920,0.871000,0.854750,0.862225
2018-08-24 14:00:00,2018-08-24,-0.579240,0.063600,0.096600,0.839400,0.853611,0.075361,0.071028,0.094789,-0.288667,...,0.036000,0.078250,0.099175,0.878667,0.871012,0.831500,0.7920,0.871000,0.854750,0.862225
